In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/TFG/Pruebas ejecucion

/content/drive/.shortcut-targets-by-id/1SmZYQ5FtbDJftz-DFi07WmU23UAjYJGL/TFG/Pruebas ejecucion


In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.7.0.84-1+cuda11.8
!pip install tensorflow keras tensorflow-estimator tensorflow-text

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.7.0.84-1+cuda11.8).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.8 MB/s eta 0:00:00


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install protobuf==3.20.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip show protobuf

Name: protobuf
Version: 3.20.3
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: google-api-core, google-cloud-bigquery, google-cloud-bigquery-storage, google-cloud-datastore, google-cloud-firestore, google-cloud-language, google-cloud-translate, googleapis-common-protos, grpcio-status, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata


In [ ]:
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
# !pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
# !pip install protobuf==3.20.3
# !pip install -q -U tensorflow-text==2.11.0 tensorflow==2.11.0

In [ ]:
from data_preprocessing import create_datasets, text_vectorization, process_text
from model import Transformer, CustomSchedule, masked_loss, masked_acc

import tensorflow as tf
from tensorflow import keras

import pandas as pd
from timeit import default_timer as timer

## Support functions

In [ ]:
train, val, test = create_datasets()
context_text_processor, target_text_processor = text_vectorization(train)

# print(context_text_processor.get_vocabulary()[:10])
# print(target_text_processor.get_vocabulary()[:10])

train_ds = train.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)
val_ds = val.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)
test_ds = val.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)

In [ ]:
def create_metrics(history, timing_callback, file_name):
    try:
        metrics = pd.read_csv('metrics/'+file_name+'.csv')
    except:
        metrics = pd.DataFrame(columns = ['loss', 'masked_acc', 'val_loss', 'val_masked_acc', 'training_time'])

    new_metrics = pd.DataFrame(history.history)
    times = timing_callback.logs  # Extraemos los valores numéricos de los tiempos de entrenamiento
    new_metrics['training_time'] = times  # Agregamos la columna 'training_time' con los valores de 'times'
    metrics = metrics.append(new_metrics, ignore_index=True)

    metrics.to_csv('metrics/'+file_name+'.csv', index = False)

In [ ]:
class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

## Model

In [ ]:
#! Model
num_layers = 6
d_model = 256
dff = 512
num_heads = 8
dropout_rate = 0.3

In [ ]:
MODEL_NAME = 'en_es_220_' + str(num_layers) + '_' + str(d_model) + '_' + str(dff) + '_' + str(num_heads) + '_' + str(dropout_rate).replace('.','-')

In [ ]:
transformer = Transformer(
    num_layers = num_layers,
    d_model = d_model,
    num_heads = num_heads,
    dff = dff,
    input_vocab_size = context_text_processor.vocabulary_size(),
    target_vocab_size = target_text_processor.vocabulary_size(),
    dropout_rate = dropout_rate,
)

## Training

In [ ]:
#! Loss and Optimizer
learning_rate = CustomSchedule(d_model)
my_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
#! Compile
transformer.compile(
    loss = masked_loss,
    optimizer=my_optimizer,
    metrics = [masked_acc],
)

In [ ]:
cb = TimingCallback()

history = transformer.fit(
    train_ds,
    epochs = 20,
    validation_data = val_ds,
    callbacks=[cb],
)

Epoch 1/10
1639/1639 [==============================] - 206s 126ms/step - loss: 3.4866 - masked_acc: 0.4944 - val_loss: 3.5118 - val_masked_acc: 0.4759
Epoch 2/10
1639/1639 [==============================] - 206s 125ms/step - loss: 3.4083 - masked_acc: 0.5075 - val_loss: 3.4706 - val_masked_acc: 0.4819
Epoch 3/10
1639/1639 [==============================] - 205s 125ms/step - loss: 3.3371 - masked_acc: 0.5188 - val_loss: 3.4612 - val_masked_acc: 0.4857
Epoch 4/10
1639/1639 [==============================] - 206s 125ms/step - loss: 3.2757 - masked_acc: 0.5290 - val_loss: 3.4164 - val_masked_acc: 0.4911
Epoch 5/10
1639/1639 [==============================] - 206s 125ms/step - loss: 3.2205 - masked_acc: 0.5377 - val_loss: 3.4327 - val_masked_acc: 0.4892
Epoch 6/10
1639/1639 [==============================] - 206s 125ms/step - loss: 3.1639 - masked_acc: 0.5461 - val_loss: 3.4081 - val_masked_acc: 0.4967
Epoch 7/10
1639/1639 [==============================] - 206s 125ms/step - loss: 3.1159 -

In [ ]:
print(cb.logs)
print(sum(cb.logs))

[205.92273340200018, 205.7035390670003, 205.39302876699958, 205.54545373400015, 205.50302829800012, 205.62217164100002, 205.65580908599986, 205.73263681900016, 205.47846882799968, 205.61476571099956]
2056.1716353529996


In [ ]:
create_metrics(history, cb, MODEL_NAME)

<ipython-input-20-fc98d706bc66>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(new_metrics, ignore_index=True)


In [ ]:
transformer.save_weights('weights/' + MODEL_NAME +  '.h5')

In [ ]:
# Save model
transformer.save('modelsFull/' + MODEL_NAME)

## Reload model

In [ ]:
# new_model = Transformer(
#     num_layers = num_layers,
#     d_model = d_model,
#     num_heads = num_heads,
#     dff = dff,
#     input_vocab_size = context_text_processor.vocabulary_size(),
#     target_vocab_size = target_text_processor.vocabulary_size(),
#     dropout_rate = dropout_rate,
# )

# new_model.compile(
#     loss = masked_loss,
#     optimizer=my_optimizer,
#     metrics = [masked_acc],
# )
# cb = TimingCallback()

# # Llama al modelo con datos ficticios para construirlo
# dummy_input = tf.ones((1, 1), dtype=tf.int32)
# dummy_context = tf.ones((1, 1), dtype=tf.int32)
# new_model((dummy_context, dummy_input), training=False)

# # Construye el optimizador con las variables del modelo
# # my_optimizer.build(new_model.trainable_variables)

# new_model.load_weights('weights/' + MODEL_NAME, + '.h5')

In [ ]:
# history = new_model.fit(
#     train_ds,
#     epochs = 3,
#     validation_data = val_ds,
#     callbacks=[cb],
# )

# create_metrics(history, cb, MODEL_NAME)
# transformer.save_weights('weights/' + MODEL_NAME, + '.h5')

In [ ]:
#@ Deprecated (use to restore the full model)

# # Save model
# transformer.save('model_en_es_0505_20')
# reconstructed_model = keras.models.load_model("model_en_es_0505_20", custom_objects={ 'CustomSchedule' : CustomSchedule, 'masked_loss': masked_loss, 'masked_acc': masked_acc })

# np.testing.assert_allclose(
#     transformer.predict(val_ds), reconstructed_model.predict(val_ds)
# )

# history = reconstructed_model.fit(
#     train_ds,
#     epochs = 3,
#     validation_data = val_ds,
#     callbacks=[cb],
# )